In [ ]:
# chunk and add all the files into chroma using api
# add all the files into chroma using api
# query the api for the passages
# construct a prompt and then add

In [ ]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [2]:
# laod json file
import json
import os

data = json.load(open('./qasper/qasper-test-v0.3.json'))
# loop through by key and value
parsed_data = []
for key, value in data.items():
    full_text = ""
    for section in value['full_text']:
        full_text += f"{section['section_name']}. "
        for paragraph in section['paragraphs']:
            full_text += f"{paragraph} "
    parsed_data.append({
        "full_text": full_text,
        "metadata": {
            "arxiv_id": key,
            "title": value['title'],
        }
    })

# write to a new json file
with open('./qasper/test-papers.json', 'w') as outfile:
    json.dump(parsed_data, outfile, indent=4)


In [9]:
# load json back into memory
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from typing import Any, List, Mapping, Optional
from enum import Enum
from langchain.llms.base import LLM

parsed_data = json.load(open('./qasper/test-papers.json'))
docs = []
for paper in parsed_data:
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  texts = text_splitter.split_text(paper['full_text'])
  docs += [Document(page_content=text, metadata=paper['metadata']) for text in texts]

# print length of docs
print(f"Number of docs: {len(docs)}")


Number of docs: 11801


In [16]:
query_prompt = """
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | gt | lt | gte | lte): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or): logical operator
- `statement1`, `statement2`, ... (comparison statements or logical operation statements): one or more statements to apply the operation to

Make sure that you only use the comparators and logical operators listed above and no others.
Make sure that filters only refer to attributes that exist in the data source.
Make sure that filters take into account the descriptions of attributes and only make comparisons that are feasible given the type of data being stored.
Make sure that filters are only used as needed. If there are no filters that should be applied return "NO_FILTER" for the filter value.

<< Example 1. >>
Data Source:
```json
{
    "content": "Lyrics of a song",
    "attributes": {
        "artist": {
            "type": "string",
            "description": "Name of the song artist"
        },
        "length": {
            "type": "integer",
            "description": "Length of the song in seconds"
        },
        "genre": {
            "type": "string",
            "description": "The song genre, one of "pop", "rock" or "rap""
        }
    }
}
```

User Query:
What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre

Structured Request:
```json
{
    "query": "teenager love",
    "filter": "and(or(eq(\"artist\", \"Taylor Swift\"), eq(\"artist\", \"Katy Perry\")), lt(\"length\", 180), eq(\"genre\", \"pop\"))"
}
```


<< Example 2. >>
Data Source:
```json
{
    "content": "Lyrics of a song",
    "attributes": {
        "artist": {
            "type": "string",
            "description": "Name of the song artist"
        },
        "length": {
            "type": "integer",
            "description": "Length of the song in seconds"
        },
        "genre": {
            "type": "string",
            "description": "The song genre, one of "pop", "rock" or "rap""
        }
    }
}
```

User Query:
What are songs that were not published on Spotify

Structured Request:
```json
{
    "query": "",
    "filter": "NO_FILTER"
}
```


<< Example 3. >>
Data Source:
```json
{
    "content": "The full text of the research paper.",
    "attributes": {
    "title": {
        "description": "The title of the research paper",
        "type": "string"
    },
    "publication_date": {
        "description": "The date the paper was published",
        "type": "date"
    },
    "arxiv_id": {
        "description": "The arxiv id of the paper",
        "type": "string"
    }
}
}
```

User Query:
What is a seed lexicon in the context of the papaer titled 'A Survey of the Usages of Deep Learning for Natural Language Processing'?

Structured Request:"""

In [32]:
# load json back into memory
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from typing import Any, List, Mapping, Optional
from enum import Enum
from langchain.llms.base import LLM

class CustomLLM(LLM):
  from langchain.callbacks.manager import CallbackManagerForLLMRun
  from langchain.llms.base import LLM

  answer = ""

  @property
  def _llm_type(self) -> str:
    return "custom"
  
  def _call(
    self,
    prompt: str,
    stop: Optional[List[str]] = None,
    run_manager: Optional[CallbackManagerForLLMRun] = None,
  ) -> str:
    import os
    import requests
    import json

    if stop is not None:
      raise ValueError("stop kwargs are not permitted.")
    answer = ""

    system_prompt = "You are a model trained on diverse datasets, you have the ability to provide insights on a wide range of topics, including machine learning."
    headers = {
      "Authorization": f"Bearer pk-SCfIRSxcfoewLTsUagmpHJMVFwVbPjHDrNeVJAbagnnEjzLD",
      "Content-Type": "application/json",
    }
    messages = [
      {
        "role": "system",
        "content": system_prompt
      },
      {
        "role": "user",
        "content": prompt
      }
    ]
    payload = {
      "model": "gpt-3.5-turbo",
      "max_tokens": 256,
      "messages": messages,
    }

    response = requests.post("https://api.pawan.krd/v1/chat/completions", headers=headers, json=payload)
    if response.status_code != 200:
      try:
          error_message = response.json()
      except ValueError:  # includes simplejson.decoder.JSONDecodeError
          error_message = response.text
      raise Exception(f"PawanGPT ERROR: {error_message} \n\n {prompt}")
    answer = response.json()['choices'][0]['message']['content']

    self.answer = answer
    return answer

In [55]:
from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain.chains.query_constructor.ir import (
    Comparator,
    Comparison,
    Operation,
    Operator,
    StructuredQuery,
    Visitor,
)
from langchain.chains.query_constructor.base import load_query_constructor_chain, AttributeInfo
from langchain.chains.query_constructor.parser import QueryTransformer
from typing import Any, Dict, List, Optional, Type, cast

query = "In the context of papers published before 2019, what is the most popular way of evluating the perfomance of question answering systems?"

llm = CustomLLM()

structured_query_translator: Visitor = ChromaTranslator()
chain_kwargs = {
    "allowed_comparators": structured_query_translator.allowed_comparators,
    "allowed_operators": structured_query_translator.allowed_operators,
}
metadata_field_info=[
    AttributeInfo(
        name="title",
        description="The title of the research paper", 
        type="string", 
    ),
    AttributeInfo(
        name="publication_date",
        description="The date the paper was published", 
        type="date", 
    ),
    AttributeInfo(
        name="arxiv_id",
        description="The arxiv id of the paper", 
        type="string", 
    ),
]
document_content_description = "The full text of the research paper."

# print(QueryTransformer)


# transformer = QueryTransformer(
#     allowed_comparators=chain_kwargs["allowed_comparators"],
#     allowed_operators=chain_kwargs["allowed_operators"],
# )

llm_chain = load_query_constructor_chain(
      llm, document_content_description, metadata_field_info, **chain_kwargs
)

inputs = llm_chain.prep_inputs(query)

structured_query = cast(
    StructuredQuery, llm_chain.predict_and_parse(callbacks=None, **inputs)
)



In [56]:
# print structured_query in a nice format
print(f"Query: {structured_query.query}")
print(f"Filters:")
for f in structured_query.filter:
  print(f"  {f}")

Query: question answering system performance evaluation
Filters:
  ('comparator', <Comparator.LT: 'lt'>)
  ('attribute', 'publication_date')
  ('value', '2019-01-01')


In [57]:
new_query, new_kwargs = structured_query_translator.visit_structured_query(structured_query=structured_query)
search_kwargs = {**new_kwargs}

print(f"New Query: {new_query}")
print(f"Search Kwargs: {search_kwargs}")

New Query: question answering system performance evaluation
Search Kwargs: {'filter': {'publication_date': {'$lt': '2019-01-01'}}}


In [ ]:
llm = CustomLLM()
response = llm(new_query)
